# MTA Data Challenges

In [91]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

In [92]:
sns.set_style('whitegrid')

**Challenge 1**

Open up a new IPython notebook
Download a few MTA turnstile data files
Open up a file, use csv reader to read it, make a python dict where there is a key for each (C/A, UNIT, SCP, STATION). These are the first four columns. The value for this key should be a list of lists. Each list in the list is the rest of the columns in a row. For example, one key-value pair should look like
{ ('A002','R051','02-00-00','LEXINGTON AVE'): [ ['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'], ['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'], ['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'], ... ] }

In [93]:
# df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_170513.txt')
def get_data(week_nums):
    # base url
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        # concat our week onto url..
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [170422, 170429, 170506, 170513]

df = get_data(week_nums)
df.columns = map(str.lower, df.columns)

In [94]:
df.columns.tolist()

['c/a',
 'unit',
 'scp',
 'station',
 'linename',
 'division',
 'date',
 'time',
 'desc',
 'entries',
 'exits                                                               ']

In [95]:
df.columns = [column.strip() for column in df.columns]

In [96]:
df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/15/2017,00:00:00,REGULAR,6136580,2078941
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/15/2017,04:00:00,REGULAR,6136613,2078947
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/15/2017,08:00:00,REGULAR,6136639,2078987
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/15/2017,12:00:00,REGULAR,6136799,2079109
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/15/2017,16:00:00,REGULAR,6137076,2079173


**Challenge 2**

Let's turn this into a time series.
For each key (basically the control area, unit, device address and station of a specific turnstile), have a list again, but let the list be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields in each list. You can convert the date and time into datetime objects -- That is a python class that represents a point in time. You can combine the date and time fields into a string and use the dateutil module to convert it into a datetime object. For an example check this StackOverflow question.

Your new dict should look something like

{ ('A002','R051','02-00-00','LEXINGTON AVE'): [ [datetime.datetime(2013, 3, 2, 3, 0), 3788], [datetime.datetime(2013, 3, 2, 7, 0), 2585], [datetime.datetime(2013, 3, 2, 12, 0), 10653], [datetime.datetime(2013, 3, 2, 17, 0), 11016], [datetime.datetime(2013, 3, 2, 23, 0), 10666], [datetime.datetime(2013, 3, 3, 3, 0), 10814], [datetime.datetime(2013, 3, 3, 7, 0), 10229], ... ], .... }

In [97]:
df['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'],format="%m/%d/%Y %H:%M:%S")

In [98]:
df['date'] = pd.to_datetime(df['date'], format="%m/%d/%Y")

In [99]:
df['time'] = pd.to_datetime(df['time'])#, format='%H:%M:%S')

In [100]:
df['time'] = df.time.dt.time

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 789772 entries, 0 to 196860
Data columns (total 12 columns):
c/a          789772 non-null object
unit         789772 non-null object
scp          789772 non-null object
station      789772 non-null object
linename     789772 non-null object
division     789772 non-null object
date         789772 non-null datetime64[ns]
time         789772 non-null object
desc         789772 non-null object
entries      789772 non-null int64
exits        789772 non-null int64
date_time    789772 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(8)
memory usage: 78.3+ MB


In [102]:
df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-15,00:00:00,REGULAR,6136580,2078941,2017-04-15 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-15,04:00:00,REGULAR,6136613,2078947,2017-04-15 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-15,08:00:00,REGULAR,6136639,2078987,2017-04-15 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-15,12:00:00,REGULAR,6136799,2079109,2017-04-15 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-15,16:00:00,REGULAR,6137076,2079173,2017-04-15 16:00:00


**Challenge 3**

* These counts are for every n hours. (What is n?) 

We want total daily entries.
Now make it that we again have the same keys, but now we have a single value for a single day, which is the total number of passengers that entered through this turnstile on this day.

In [104]:
df['daily_entries'] = df.groupby(['c/a', 'unit', 'scp', 'station'])['entries'].diff()

ValueError: 

In [ ]:
df.head()

In [ ]:
def clean_entries(row, max_counter):
    each = row["daily_entries"]
    if each < 0:
        each = -each
    if each > max_counter:
        return 0
    return each

df['daily_entries'] = df.apply(clean_entries, axis=1, max_counter=10000)

In [ ]:
df.daily_entries.isnull().sum()

In [ ]:
df.dropna(subset=["daily_entries"], axis=0, inplace=True)

In [ ]:
df.head()

**Challenge 4**

We will plot the daily time series for a turnstile.

Take the list of [(date1, count1), (date2, count2), ...], for the turnstile and turn it into two lists: dates and counts. This should plot it:

```python
plt.figure(figsize=(10,3))
plt.plot(dates,counts)
```

In [ ]:
stationMask = ((df["c/a"] == "A002") & 
(df["unit"] == "R051") & 
(df["scp"] == "02-00-00") & 
(df["station"] == "59 ST"))

In [ ]:
sampleStation = df[stationMask]

In [ ]:
sampleStation.shape

In [ ]:
dateGroup = sampleStation.groupby('date').daily_entries.sum()

In [ ]:
dateGroup.plot()

**Challenge 5**

- So far we've been operating on a single turnstile level, let's combine turnstiles in the same ControlArea/Unit/Station combo. 

There are some ControlArea/Unit/Station groups that have a single turnstile, but most have multiple turnstilea-- same value for the C/A, UNIT and STATION columns, different values for the SCP column.
We want to combine the numbers together -- for each ControlArea/UNIT/STATION combo, for each day, add the counts from each turnstile belonging to that combo.

In [ ]:
# mask2 = ((df["c/a"] == "A002") & 
# (df["unit"] == "R051") &  
# (df["station"] == "59 ST"))

In [ ]:
cusDay = df.groupby(['c/a','unit','station','date']).daily_entries.sum().reset_index()

In [ ]:
cusDay.shape

In [ ]:
cusDay.head()

**Challenge 6**

Similarly, combine everything in each station, and come up with a time series of [(date1, count1),(date2,count2),...] type of time series for each STATION, by adding up all the turnstiles in a station.

In [ ]:
stationDaily = cusDay.groupby(['station','date']).daily_entries.sum().reset_index()

In [ ]:
stationDaily.shape

In [ ]:
stationDaily.head()

**Challenge 7**

Plot the time series for a station.

In [ ]:
sampleStation = stationDaily[stationDaily.station == '59 ST']

In [ ]:
sampleGroup = sampleStation.groupby('date').daily_entries.sum()

In [ ]:
sampleGroup

In [ ]:
sampleGroup.plot()
plt.title('59 ST')
plt.ylabel('Daily Entries')
plt.xlabel('Date')

**Challenge 8**

- Make one list of counts for one week for one station. Monday's count, Tuesday's count, etc. so it's a list of 7 counts. Make the same list for another week, and another week, and another week. plt.plot(week_count_list) for every week_count_list you created this way. You should get a rainbow plot of weekly commute numbers on top of each other.

In [ ]:
df['week'] = df.date_time.dt.weekofyear

In [ ]:
df.head()

In [ ]:
df['day'] = df.date_time.dt.dayofweek

In [ ]:
df.head()